# Pre-processing DNA and protein sequences for Machine Learning applications
Machine Learning algorithms can be used to process, classify or generate text in a very efficient and large-scale way. Similarly, these ML algorithms could also be applied to raw DNA or protein sequence data. However, due to inherent differences between genetic code and human written text, we need to first pre-process the sequences into a proper format that could be fed into ML algorithms such as, Convolutional or Recurrent Neural Networks (CNN or RNN). One such popular approach is to extract k-mers from raw sequences and treat them as equivalent to words within a human written text. In this python Jupyter notebook, I'll explain how to read sequences from one or multiple fasta files, extract k-mers, map k-mers to vectors of real numbers (word embedding) and save the reformatted data.

First things first, we start with loading the required packages.

In [1]:
import numpy as np
import os
from six.moves import cPickle

Raw DNA or protein sequences usually come in the form a fasta file containing multiple sequences. The following function helps with reading the sequences from a fasta file, one by one.

In [2]:
# A function to read sequences from fasta files
def read_fasta(fp):
    name, seq = None, []
    for line in fp:
        line = line.rstrip()
        if line.startswith(">"):
            if name: yield (name.replace(">", ""), ''.join(seq))
            name, seq = line, []
        else:
            seq.append(line)
    if name: yield (name.replace(">", ""), ''.join(seq))

Next we need a function to extract k-mers from raw sequences. The following function accepts a sequence and the desired length of the k-mers (or _k_) and returns the list of k-mers extracted from the sequence.

In [3]:
# A function to extract k-mers from a sequence
def seq2kmer(seq, k):
    kmers = zip(*[seq[i:] for i in range(k)])
    return [''.join(x) for x in list(kmers)]

We'll also need a function to write the k-mers and their embeddings into files. The vocabulary file (consisting of the unique k-mers) will be written in Pickle format. The k-mers, their class and their embeddings as real value vectors will be written to a csv file.

In [4]:
def write_files(outfile, all_kmers, unique_kmers, seqids):
    vocab_size = len(unique_kmers)
    vocab = dict(zip(unique_kmers, range(len(unique_kmers))))
    tensor = np.array([list(map(vocab.get, k)) for k in all_kmers])
    # Write vocabulary to a cPickle file
    with open(outfile + "_vocab.pkl", 'wb') as IN:
        cPickle.dump(unique_kmers, IN)
    
    with open(outfile + ".csv", "w") as OUT:
        for r in range(len(seqids)):
            OUT.write("{}\t{}\t{}\n".
                      format(seqids[r],
                             " ".join(all_kmers[r]),
                             " ".join(map(str, tensor[r]))))

The following function takes the fasta file and creates the vocabulary and word embeddings using the above functions.

In [5]:
def seq_processor(infile, k):
    all_kmers = []
    unique_kmers = []
    seqids = []
    with open(infile, "r") as fp:
        for name, seq in read_fasta(fp):
            seqids.append(name)
            kmers = seq2kmer(list(seq), k)
            all_kmers.append(kmers)
            unique_kmers += [x for x in kmers if x not in unique_kmers]
    return [all_kmers, unique_kmers, seqids]

Now, to run the code on a toy fasta file to find k-mers of the length 5bp in raw DNA sequences. We read the file from the "raw" folder and write data to "processed" folder.

In [6]:
infile = "data/raw/file1.fa"
outfile = "data/processed/file1"
k = 5
all_kmers, unique_kmers, seqids = seq_processor(infile, k)
write_files(outfile, all_kmers, unique_kmers, seqids)

In case of multiple fasta files. To be continued...